## 04 - part 3 - Filtering in the frequency domain

In [1]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from numpy import fft

In [108]:
def low_pass_filter(n, m, r):
    filter = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            filter[i][j] = 1 if np.sqrt((i - n/2.0)**2 + (j - m/2.0)**2) <= r else 0
    return filter

In [3]:
def high_pass_filter(n, m, r):
    filter = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            filter[i][j] = 1 if np.sqrt((i - n/2)**2 + (j - m/2)**2) > r else 0
    return filter

In [4]:
def band_stop_filter(n, m, r1, r2):
    filter = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            d = np.sqrt((i - n/2)**2 + (j - m/2)**2)
            filter[i][j] = 0 if d >= r1 and d <= r2  else 1
    return filter

In [5]:
def laplacian_filter(n, m):
    filter = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            filter[i][j] = -4 * (np.pi**2) * ((i - n/2)**2 + (j - m/2)**2)
    return filter

In [126]:
def gaussian_filter(n, m, sigma1, sigma2):
    filter = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            filter[i][j] = np.e ** -(((i - n/2)**2 / ((2 * sigma1**2)) + (j - m/2)**2 / (2 * sigma2**2)))
    return filter        

In [7]:
def read_image(image_name):
    return imageio.imread(f"images/{image_name}")

In [8]:
def print_image(image):
    plt.figure(figsize=(10,10)) 
    plt.imshow(image, cmap="gray")
    plt.axis('off') # remove axis with numbers

In [9]:
def root_mean_squared_error(result_image, reference_image):
    n = result_image.shape[0]
    sum = 0
    for i in range(n):
        for j in range(n):
            sum += (result_image[i][j] - reference_image[i][j]) ** 2
    return np.round((np.sqrt(sum))/n, 4)


# print_image(reference_image)

In [11]:
def get_filter_from_input(image_n, image_m):
    index = int(input().rstrip())
    match index:
        case 0: # low-pass filter
            r = int(input().rstrip())
            return low_pass_filter(image_n, image_m, r)
        case 1: # high-pass
            r = int(input().rstrip())
            return high_pass_filter(image_n, image_m, r)
        case 2: # band-stop
            r0 = int(input().rstrip())
            r1 = int(input().rstrip())
            return band_stop_filter(image_n, image_m, r0, r1)
        case 3: # laplacian
            return laplacian_filter(image_n, image_m)
        case 4: # gaussian
            s0 = int(input().rstrip())
            s1 = int(input().rstrip())
            return gaussian_filter(image_n, image_m, s0, s1)

In [113]:
def filter_fft_image(image, filter):
    fourier_domain = fft.fft2(image)
    fourier_filtered = fft.fftshift(fourier_domain) * filter
    # fourier_filtered = fourier_domain * filter
    filtered_image = fft.ifft2(fft.ifftshift(fourier_filtered))
    # filtered_image = fft.ifft2(fourier_filtered)
    real_image = np.real(filtered_image)
    result = (real_image - np.min(real_image)) * (255 / (np.max(real_image) - np.min(real_image)))
    return result


In [127]:
base_image_name = "apollo17.png"
reference_image_name = "apollo17_ref-case9.png"
index = 0
radius = 20
sigma1 = 60
sigma2 = 30

In [128]:
base_image = read_image(base_image_name)
reference_image = read_image(reference_image_name)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_10964\1778181336.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(f"images/{image_name}")


In [129]:
result_image = filter_fft_image(base_image, gaussian_filter(base_image.shape[0], base_image.shape[1], sigma1, sigma2))

In [130]:
root_mean_squared_error(result_image, reference_image)

0.5769